**_*Note:_** **_Always use venv or Anaconda Navigator to avoid packages conflict._**

# Step 1: Installing required packages/softwares

In [2]:
#pip install django
#pip install mssql-django

#Uncomment if needed

**Also install:**
- _SQL Server Express_
- _SQL Server Management Studio_
- _ODBC Driver 17 for SQL Server_

# Step 2: Creating a Django Project

In [ ]:
django-admin startproject myproject
#This will create a project folder named "myproject". You can change the name accordingly.

cd myproject
#Navigate into the project folder

# Step 3: Understand Project Structure
#### In directory, you’ll see the following files and folders:
```
myproject/
    manage.py
    myproject/
        __init__.py
        settings.py
        urls.py
        asgi.py
        wsgi.py
```

- _**manage.py:** A command-line utility for interacting with the project._

- _**myproject/settings.py:** Configuration settings for the project (e.g., database, installed apps, etc.)._

- _**myproject/urls.py:** URL routing for the project._


# Step 4: Configure the Database
_*Note: Make sure you have created the server using SSMS._
1. Open myproject/settings.py
2. Find the DATABASES section and update it to use mssql-django with your server configuration:
```
DATABASES = {
    'default': {
        'ENGINE': 'mssql',
        'NAME': 'your_database_name',  # Replace with your database name
        'USER': 'your_username',       # Replace with your SQL Server username
        'PASSWORD': 'your_password',   # Replace with your SQL Server password
        'HOST': 'your_server_address', # Replace with your SQL Server host (e.g., 'localhost' or an IP address)
        'PORT': '',                    # Leave empty for default port
        'OPTIONS': {
            'driver': 'ODBC Driver 17 for SQL Server',  # Ensure this driver is installed
            'Trusted_Connection':'yes', # Add this when using local server
        },
    }
}
```
3. Save the file.


# Step 5: Creating a Django App

In [ ]:
python manage.py startapp products
#You can change the name accordingly.

#### This will create a folder named "products" with the following structure:
```
products/
    migrations/
    __init__.py
    admin.py
    apps.py
    models.py
    tests.py
    views.py
```

### Registering the defined app:

In [ ]:
#Open myproject/settings.py and add 'products'app to the INSTALLED_APPS list:

INSTALLED_APPS = [
    ...
    "products.apps.ProductsConfig",  #Add this at the end of the list. Change name accordingly.
]

# Step 6: Defining Models

Models define the structure of database tables. Here we make two tables for "Product" and "Category".

_*Note: Here, the Product model will have a foreign key relationship to the Category model._

In [ ]:
#Open products/models.py and define the models as follows: (Define according to your project)

class Category(models.Model):
    Id = models.AutoField(primary_key=True)
    Name = models.CharField(max_length=100)
    Description = models.TextField()
    Deleted = models.BooleanField(default=False)  # Soft delete flag

    def __str__(self):
        return self.Name

class Product(models.Model):
    Id = models.AutoField(primary_key=True)
    Name = models.CharField(max_length=100)
    ProductImage = models.TextField()
    Description = models.TextField()
    Price = models.DecimalField(max_digits=10, decimal_places=2)
    Stock = models.IntegerField()
    CreatedAt = models.DateTimeField(auto_now_add=True)
    Deleted = models.BooleanField(default=False)  # Soft delete flag
    CategoryID = models.ForeignKey(Category, on_delete=models.CASCADE)

    def __str__(self):
        return self.Name

#### Here we have:

**Category Model:**

- _Name: Name of the category._

- _Description: Description of the category._

- _Deleted: A flag to indicate if the category is soft-deleted (default is False)._

**Product Model:**

- _Name: Name of the product._

- _Price: Price of the product._

- _Stock: Quantity of the product in stock._

- _Created at: Timestamp when the product was created._

- _Deleted: A flag to indicate if the product is soft-deleted (default is False)._

- _Category: Foreign key to the Category model, establishing a relationship between products and categories._

# Step 7: Create and Apply Migrations
 This is to create the corresponding tables in SQL Server database.

In [ ]:
#In project directory run:
python manage.py makemigrations

#This generates migration files in the products/migrations/ folder.

In [ ]:
#In directory run:
python manage.py migrate

#This creates the tables (Category and Product) in your SQL Server database.

### Verify the Tables:
1. Open your SQL Server Management Studio (SSMS) or any SQL Server client.
2. Check your database. You should see two new tables:
    - products_category
    - products_product

_* You can see the tables by right-clicking on Database Diagrams>selecting New Database Diagram>selecting the two table>Add_

![alt text](<Screenshot 2025-01-16 200055-1.png>)

# Step 8: Use Django Admin to Manage Data
Django provides an admin interface to manage your data.

In [ ]:
#Create a superuser:
python manage.py createsuperuser

#You can Use appropriate information to fill.

### Registering the defined models:

In [ ]:
#Open products/admin.py and add the following:

from django.contrib import admin
from .models import Category, Product

admin.site.register(Category)
admin.site.register(Product)

In [ ]:
#Run the server:

python manage.py runserver

_*Access the admin interface at http://127.0.0.1:8000/admin/ and log in with your superuser credentials._

You can now add, edit, and delete categories and products.

# Step 9: Create Views and Templates
Optional. To display products on a webpage.

### Create a view:

In [ ]:
#Open products/views.py and add the following:

from django.shortcuts import render
from .models import Product

def product_list(request):
    products = Product.objects.all()
    return render(request, 'products/product_list.html', {'products': products})

### Create a template:

In [ ]:
#Create a folder named templates/products/ inside the products app. Then, create a file named product_list.html:

<!DOCTYPE html>
<html>
<head>
    <title>Product List</title>
</head>
<body>
    <h1>Product List</h1>
    <ul>
        {% for product in products %}
            <li>{{ product.Name }} - ${{ product.Price }} ({{ product.Category.Name }})</li>
        {% endfor %}
    </ul>
</body>
</html>

### Configure URLs:

In [ ]:
#Open products/urls.py (create it if it doesn’t exist) and add the following:

from django.urls import path
from . import views

urlpatterns = [
    path('', views.product_list, name='product_list'),
]

In [ ]:
#Then, include the products URLs in myproject/urls.py:

from django.contrib import admin
from django.urls import include, path

urlpatterns = [
    path('admin/', admin.site.urls),
    path('products/', include('products.urls')),
]


In [ ]:
#Run the server:

python manage.py runserver

_Visit http://127.0.0.1:8000/products/ to see your product list._

# Step 10: Django ORM (Object-Relational Mapping): 
Performing CRUD (Create, Read, Update, Delete) operations using Django ORM.